# MarathoNet

_MarathoNet_ es un sistema inteligente capaz de detectar el número de dorsal de corredores de maratón a partir del video extraído de una cámara que graba la carrera en un punto de control.

## Descripción del problema

Para poder identificar el dorsal de un corredor necesitamos detectar en tiempo real dónde se encuentra el cuerpo de éste en cada frame que extraemos de un vídeo que graba la carrera. 

En este notebook se plantea utilizar la librería _DLIB_ preentrenado con un modelo para la identificación de puntos clave de la cara. Los cuales podrán ser de utilidad para diseñar un subsistema de identificación y tracking de corredores de maratón.

## Pruebas iniciales con _wrnchAI_

Con el fin de familiarizarnos con la librería [_wrnchAI_](http://dlib.net/), disponemos de una serie de vídeos de personas moviéndose en una escena que utilizaremos como toma de contacto inicial con la librería y así observar todas la potencia y funcionalidades que esta ofrece.